In [9]:
import os
import pandas as pd
import pickle
from utils import *
from tqdm import tqdm

In [2]:
# read dataset
movie_path = 'dataset\movies.pickle'
with open(movie_path, 'rb') as mov:
    movies = pickle.load(mov)
    
# movies

In [3]:
# generate prompts
entities = {}
prompts = {}
for entity in movies:
    title = tuple(entity[CONTEXTUALISING_ATTRIBUTES].items())[0][1] # get the value of the first contextrualising attribute (movie title)
    prompt = f"What is the full name of the {entity['entity']} with the following associated information?:"
    for concept, value in tuple(entity[CONTEXTUALISING_ATTRIBUTES].items())[1:]:
        prompt += f"\n- {concept.replace('_',' ')}: {value}"
    for concept_class in entity[TARGET_ATTRIBUTES]:
        for concept, value in entity[TARGET_ATTRIBUTES][concept_class].items():
            prompt += f"\n- {concept.replace('_',' ')}: {value}"
    prompts[title] = prompt
    entities[title] = entity

# sample movie+prompt, for demonstration
print("Entity name:", list(prompts.items())[0][0])
print("Prompt:", list(prompts.items())[0][1])

Entity name: The Shawshank Redemption
Prompt: What is the full name of the movie with the following associated information?:
- Released Year: 1994
- Genre: Drama
- IMDB Rating: 9.3
- No of Votes: 2343110
- Certificate: A
- Director: Frank Darabont
- Star1: Tim Robbins
- Star2: Morgan Freeman
- Star3: Bob Gunton
- Star4: William Sadler


In [4]:
# evaluation function
def entity_scorer (generated:str, target:str):
    """
    Scorer function, comparing the generated text to the target.
    Returns True if the generated matches the target, False otherwise
    """
    if target.lower() in generated.lower():
        return True
    return False

In [5]:
# check previously verified entities
try:
    verified_movie_path = 'dataset\movies_verified.pickle'
    with open(verified_movie_path, 'rb') as ver:
        verified = pickle.load(ver)
except FileNotFoundError:
    verified = []
    
verified_titles = [tuple(entity[CONTEXTUALISING_ATTRIBUTES].items())[0][1] for entity in verified]

In [11]:
# query model - this does not use the logger at all
model='gpt-4-0314'
temp=0

missed = []
prompt_list = list(prompts.items())[:15]
pbar = tqdm(total=len(prompt_list))
for title, prompt in prompt_list:
    if title in verified_titles: # do not repeat entities
        print("Already verified:", title)
        pbar.update(1)
        continue
    messages = [{'role':'user', 'content':prompt}]
    response = chatgpt_query(messages, model=model, temperature=temp)
    if entity_scorer(response, title):
        verified.append(entities[title])
        verified_titles.append(title)
    else:
        missed.append(entities[title])
    pbar.update(1)
pbar.close()
print("# entities correctly identified:", len(verified), "out of", len(prompt_list))

with open('dataset/movies_verified.pickle', 'wb') as handle:
    pickle.dump(verified, handle)

  0%|          | 0/15 [00:00<?, ?it/s]

Already verified: The Shawshank Redemption
Already verified: The Godfather
Already verified: The Dark Knight
Already verified: The Godfather: Part II
Already verified: 12 Angry Men
Already verified: The Lord of the Rings: The Return of the King
Already verified: Pulp Fiction
Already verified: Schindler's List
Already verified: Inception
Already verified: Fight Club


 73%|███████▎  | 11/15 [00:01<00:00,  5.62it/s]

{
  "id": "chatcmpl-80HKf1mfAZXg9iRogbQ1cgAdYo9av",
  "object": "chat.completion",
  "created": 1695077317,
  "model": "gpt-4-0314",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The full name of the movie is The Lord of the Rings: The Fellowship of the Ring."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 98,
    "completion_tokens": 19,
    "total_tokens": 117
  }
}


 80%|████████  | 12/15 [00:02<00:00,  3.89it/s]

...

{
  "id": "chatcmpl-80HKhNlEPxbfisOlXomEMT4Eoa0Rx",
  "object": "chat.completion",
  "created": 1695077319,
  "model": "gpt-4-0314",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The full name of the movie is \"The Good, the Bad and the Ugly\" (Italian: Il buono, il brutto, il cattivo)."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 100,
    "completion_tokens": 34,
    "total_tokens": 134
  }
}


 93%|█████████▎| 14/15 [00:08<00:00,  1.06it/s]

{
  "id": "chatcmpl-80HKlbMcyyfXPS5DYK9zU8EYSRBDe",
  "object": "chat.completion",
  "created": 1695077323,
  "model": "gpt-4-0314",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The full name of the movie is The Lord of the Rings: The Two Towers."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 100,
    "completion_tokens": 17,
    "total_tokens": 117
  }
}


100%|██████████| 15/15 [00:09<00:00,  1.61it/s]

{
  "id": "chatcmpl-80HKnijjGQvg1RA1hJBjYF6Ur6WAR",
  "object": "chat.completion",
  "created": 1695077325,
  "model": "gpt-4-0314",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The Matrix"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 107,
    "completion_tokens": 2,
    "total_tokens": 109
  }
}
# entities correctly identified: 15 out of 15
